# nanoGPT

本教程基于 [nanoGPT](https://github.com/karpathy/nanoGPT) 进行中型 GPT 的训练和微调。

## 环境准备

> 建议使用 [PyTorch 2.0](https://pytorch.org/get-started/pytorch-2.0) 环境运行本教程。PyTorch 2.0 增加了 `torch.compile()`，通过编译的方式，仅用一行代码即可实现模型的稳定加速。

在终端中运行以下命令，可进行 PyTorch 2.0 的安装：

```shell
pip3 install --user --pre torch torchvision torchaudio --force-reinstall --index-url https://download.pytorch.org/whl/nightly/cu117
```

在终端中运行以下命令安装其他依赖：

```shell
pip install -r requirements.txt
```

## 模型训练

此过程默认复现 OpenAI 发布的最小 GPT-2 模型（即 124M 版本）。

### 数据准备

模型训练使用来自 OpenAI 的 WebText 数据集（12.9G）。

在终端中依次运行下列命令，将文档 tokenize 为一个简单的长一维索引数组：

```shell
cd data/openwebtext
python prepare.py
```

这将生成两个文件：train.bin 和 val.bin，每个文件都包含一个代表 GPT-2 BPE token id 的 uint16 字节原始序列。

> **注意：** `prepare.py` 脚本运行所需时间较长，我们已将生成好的 train.bin 和 val.bin 文件添加至 [openwebtext 数据集](https://openbayes.com/console/open-tutorials/datasets/5iZ23a5zCGe/1/overview)，以便在后续执行中快速载入数据。此容器已绑定 openwebtext 数据集至 `/openbayes/input/input0` 目录下。您可以跳过『数据准备』步骤直接进行训练。

### 训练

在终端中依次运行下列命令：

```shell
# 返回 /openbayes/home 目录
cd /openbayes/home

# 开始训练
python train.py
```


默认情况下，会定期向输出目录写入 checkpoints。只要有一个 checkpoints 后，就可以运行下列命令从模型中进行采样：

```shell
python sample.py
```

部分参数说明：
- out_dir: checkpoints 保存路径，默认为 "out"
- start: 生成样本的起始文本
- num_samples: 生成的样本数量，默认为 10
- max_new_tokens: 每个样本生成的最大 token 数量，默认为 500

如图为训练 96000 个 step 后，模型生成的文本样例：
![sample](show/sample_96000.png)

## 模型训练 demo

若没有足够的时间和资源，仍想体验模型训练过程，可参考此过程训练一个 toy model。

### 数据准备

使用莎士比亚数据集。

在终端中依次运行下列命令：

```shell
cd data/shakespeare
python prepare.py
```

> **注意：** 此容器已绑定了处理好的 [shakespeare 数据集](https://openbayes.com/console/open-tutorials/datasets/ykDI9iRlxZb/1/overview)至 `/openbayes/input/input1` 目录下。您可以跳过『数据准备』步骤直接进行训练。

### 训练

在终端中依次运行下列命令：

```shell
# 返回 /openbayes/home 目录
cd /openbayes/home

# 开始训练
python train.py \
       --dataset=shakespeare \
       --n_layer=4 \
       --n_head=4 \
       --n_embd=64 \
       --eval_iters=1 \
       --block_size=64 \
       --batch_size=8 \
       --max_iters=10000
```

部分参数说明：

- out_dir: checkpoints 保存路径，默认为 "out"
- dataset: 数据集
- n_layer: 神经网络的层数
- n_head: 注意力头的数量
- n_embd: 词向量、位置向量以及内部特征向量的维数
- eval_iters: 评估迭代次数
- block_size: 块大小
- batch_size: 批大小
- max_iters: 训练迭代次数
- device: 默认为 "cuda"，若在 CPU 上运行则需设置为 "cpu"
- compile: 使用 PyTorch2.0 中的 `torch.compile()` 加速模型训练，默认为 "True"，若在 PyTorch < 2.0 环境中运行，则需设置为 "False"

![compare](show/compare.jpg)
> **注意：** 在 PyTorch2.0 环境中使用 torch.compile() 进行训练相比在 PyTorch1.13 环境中，单次迭代所需的时间将减少一半以上。

## 微调

基于 GPT 模型进行微调。

本教程中使用莎士比亚数据集，基于 `gpt2-xl` 模型进行微调。

可参考『模型训练 demo』进行数据准备，然后在终端中运行下列命令：

```shell
python train.py finetune_shakespeare.py
```

部分参数说明：
- init_from: 初始化模型，可选参数：'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'，默认为 'gpt2-xl'。

### 模型推理

在终端中运行以下命令即可生成莎士比亚风格的文本：

```shell
python sample.py --out_dir=out-shakespeare
```

如图为微调 1000 个 step 后，模型生成的文本样例：

![finetune_1000](show/finetune_1000.png)

通过 `start` 参数，控制模型以 “To be or not to be” 为开头生成后续文本：

```shell
python sample.py --out_dir=out-shakespeare --start="To be or not to be"
```

![finetune_start](show/finetune_start.png)